In [22]:
import torch
import numpy as np

In [1]:
from abc import abstractclassmethod, ABC
from dataclasses import dataclass, replace

@dataclass
class Metadata():
    encoding: str

@dataclass
class BarMetadata(Metadata):
    other: str = ""

class Foo(ABC):
    def __init__(self, encoding: str):
        self.metadata: Metadata = Metadata(encoding)


    def load_metadata():
        pass

    @abstractclassmethod
    def test(self, file: str):
        file += "   padding"

class Bar(Foo):
    def __init__(self, encoding: str):
        super().__init__()
        self.metadata: BarMetadata

    def test(self, file: str):
        super().test(file)
        print(file)

In [24]:
from dataclasses import dataclass
@dataclass
class Metadata():
    encoding: str

@dataclass
class BarMetadata(Metadata):
    other: str = ""

In [30]:
test = BarMetadata(encoding="gpt2", other="ok")
import dataclasses
dataclasses.replace(test, **{"other": "Bruh"})

BarMetadata(encoding='gpt2', other='Bruh')

In [4]:
test: Metadata = Metadata("gpt2")
test: BarMetadata = BarMetadata(**test, other="other")

TypeError: __main__.BarMetadata() argument after ** must be a mapping, not Metadata

In [20]:
obj = test
params = set(inspect.signature(Metadata.__init__).parameters.keys()) - set(['self'])
{x:getattr(obj, x) for x in params}

{'encoding': 'gpt2'}

In [22]:
from dataclasses import asdict, 
asdict(test)

{'encoding': 'gpt2'}

In [2]:
# padding does not get appended to the parameter as it is a seperate function
Bar().test("test")

test


In [4]:
b,c,e = 4, 5,6
tensor_3d = torch.arange(b*c*e).reshape(b,c,e)

In [5]:
tensor_3d

tensor([[[  0,   1,   2,   3,   4,   5],
         [  6,   7,   8,   9,  10,  11],
         [ 12,  13,  14,  15,  16,  17],
         [ 18,  19,  20,  21,  22,  23],
         [ 24,  25,  26,  27,  28,  29]],

        [[ 30,  31,  32,  33,  34,  35],
         [ 36,  37,  38,  39,  40,  41],
         [ 42,  43,  44,  45,  46,  47],
         [ 48,  49,  50,  51,  52,  53],
         [ 54,  55,  56,  57,  58,  59]],

        [[ 60,  61,  62,  63,  64,  65],
         [ 66,  67,  68,  69,  70,  71],
         [ 72,  73,  74,  75,  76,  77],
         [ 78,  79,  80,  81,  82,  83],
         [ 84,  85,  86,  87,  88,  89]],

        [[ 90,  91,  92,  93,  94,  95],
         [ 96,  97,  98,  99, 100, 101],
         [102, 103, 104, 105, 106, 107],
         [108, 109, 110, 111, 112, 113],
         [114, 115, 116, 117, 118, 119]]])

In [23]:
#batch has 5 rows, only want 3 
index = torch.tile(torch.arange(3).reshape(3,1), (b,1,e))
#you only consider the first batch
torch.gather(tensor_3d, dim=1, index=index)

tensor([[[  0,   1,   2,   3,   4,   5],
         [  6,   7,   8,   9,  10,  11],
         [ 12,  13,  14,  15,  16,  17]],

        [[ 30,  31,  32,  33,  34,  35],
         [ 36,  37,  38,  39,  40,  41],
         [ 42,  43,  44,  45,  46,  47]],

        [[ 60,  61,  62,  63,  64,  65],
         [ 66,  67,  68,  69,  70,  71],
         [ 72,  73,  74,  75,  76,  77]],

        [[ 90,  91,  92,  93,  94,  95],
         [ 96,  97,  98,  99, 100, 101],
         [102, 103, 104, 105, 106, 107]]])

In [18]:
#objective: retrieve first rows of tensor_3d -> if we specify dim=1, we collapse along the rows (we perform indexing for each row)
#b,c,e = 4,5,6
#i always want the first row -> specify by row, dim=1
#how do i reduce the amount of rows if the index tensor has to be of the same dimension?
#dimension has to be the same but not the shape
#torch.zeros(4,1,6) gets the first row of the tensor, but it is problematic if i want multiple rows as i 
#then use the same index (0) while having the output shape that i want
#solution: arange
#index=torch.zeros(4,1,6) -> if we use 5 instead of 6, each row has 5 columns
#meaning: we need a row containing the same index 
tensor_3d.gather(dim=1, index=torch.zeros(4,2,6, dtype=torch.int64))

tensor([[[ 0,  1,  2,  3,  4],
         [ 0,  1,  2,  3,  4]],

        [[30, 31, 32, 33, 34],
         [30, 31, 32, 33, 34]],

        [[60, 61, 62, 63, 64],
         [60, 61, 62, 63, 64]],

        [[90, 91, 92, 93, 94],
         [90, 91, 92, 93, 94]]])

In [20]:
torch.arange(2).reshape(2,1)

tensor([[0],
        [1]])

In [6]:
y = torch.tensor([
     [
       [1, 2, 3],
       [4, 5, 6],
       [0, 0, 0],
       [0, 0, 0]
     ],
     [
       [1, 2, 3],
       [4, 5, 6],
       [0, 0, 0],
       [0, 0, 0]
     ],
     [
       [1, 2, 3],
       [4, 5, 6],
       [0, 0, 0],
       [0, 0, 0]
     ]
   ])
#size is: 3, 4, 3. if you collapse in the first dimension (dim=0), the result tensor becomes of size 4,3. if you collapse it in the second dimension, you get a tensor of size 3,3

In [8]:
y.sum(dim=1)
#in transformers, we usually have tensors of shape b,c,e (batch_size, context, embedding_dimension).
#if we specify dim=0, we perform the operation along the entire batch, in dim=1 along the context and in dim=2 along the embedding dimension.
#if we were to sum the tensors together, sum(dim=1) will yield the sum of the embeddings of each word.
#think of it as squishing a dimension together so that it is of size 1, meaning that we have to squeeze in that dimension.

tensor([[5, 7, 9],
        [5, 7, 9],
        [5, 7, 9]])

In [ ]:
#batchnorm performed with padding
[-0.9041, -1.0757,  0.5234, -0.0512,  0.6242, -0.1665, -1.0276,
           1.6814, -0.9740, -1.8245, -0.3201,  0.7656,  0.2983,  0.1673,
           0.2048,  2.0786],
[-0.8093,  0.6853, -1.1480, -1.3283,  1.0316,  1.3167, -0.2717,
          -1.2307,  1.8153, -0.4639,  1.2193, -0.7859, -0.2366, -0.7097,
           1.0948, -0.1788],
[ 0.8650, -1.5553,  0.7642, -1.4681,  0.5856, -1.0539,  0.5444,
           0.3433, -1.5597,  0.4344,  1.2369, -0.9978, -0.2923,  1.5946,
          -0.1032,  0.6620],
#without padding
[-0.9041, -1.0757,  0.5234, -0.0512,  0.6242, -0.1665, -1.0276,
           1.6814, -0.9740, -1.8245, -0.3201,  0.7656,  0.2983,  0.1673,
           0.2048,  2.0786],
[-0.8093,  0.6853, -1.1480, -1.3283,  1.0316,  1.3167, -0.2717,
          -1.2307,  1.8153, -0.4639,  1.2193, -0.7859, -0.2366, -0.7097,
           1.0948, -0.1788],
[ 0.8650, -1.5553,  0.7642, -1.4681,  0.5856, -1.0539,  0.5444,
           0.3433, -1.5597,  0.4344,  1.2369, -0.9978, -0.2923,  1.5946,
          -0.1032,  0.6620]
#-> padding does not artificially lower mean in 3d inputs, in 2d inputs they are lowered though
#as normalization is performed along the embeddings
#larger batches will calculate the mean of the words of multiple sentences
tensor([[[-0.5822,  1.3051,  0.5337, -0.3419, -0.5721,  1.6196, -0.0168,
           0.5965,  0.3864, -1.1112, -1.4737,  1.6359, -0.6583, -1.1196,
          -0.3974,  0.9116],
         [ 1.2417,  1.5342, -1.2204,  0.7292,  0.8772, -1.2491, -1.8027,
           0.5878, -0.2417, -1.0617, -0.0714, -1.5258,  0.6894, -0.6110,
           1.4409,  0.8721],
         [ 1.2212,  1.5189,  0.8535,  0.5534,  1.3774, -0.5945, -0.4424,
          -0.3580,  0.1088, -1.7387,  0.1289, -0.2090,  0.1482, -1.7862,
          -1.1194,  0.3945],
         [ 0.0922,  1.5306, -0.4437,  0.8035, -0.0281, -0.0908,  0.4842,
           1.1845, -1.6692,  0.9497,  1.3172, -0.6635, -0.4553,  0.3977,
          -1.3318,  0.6270],
         [ 0.0024,  0.5799,  1.7395, -1.0177,  0.6052, -1.6885,  0.0204,
           0.1526,  1.2536, -1.1935,  1.7002,  0.2616, -1.5268, -0.8731,
          -1.0120, -0.7853],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000]],

        [[ 1.1908,  1.0052, -0.8598,  1.0370,  1.3233, -1.2132, -0.3867,
          -0.7785, -1.3937, -0.0684,  1.4887, -0.2831,  1.5336,  1.6622,
           1.3477, -0.2572],
         [-0.1492, -0.6825,  0.4371,  1.0500,  1.9715,  0.0609,  1.1558,
          -1.1479,  0.6885,  1.6560, -0.3192,  1.6064, -0.3162, -0.9805,
          -1.9670,  1.6830],
         [ 1.2578,  0.2911, -1.3203,  0.3955, -0.0091,  0.9071,  0.6828,
          -1.6579, -0.4551,  0.9677,  0.6999, -0.1291,  0.3606, -1.3197,
          -0.9213,  0.7788],
         [ 0.3692, -0.9861, -1.7729,  0.2044,  1.0779, -1.5686, -0.0076,
          -1.0156, -0.1939,  0.0496,  0.3885, -1.6978,  0.0456,  1.6219,
           1.2923, -0.0506],
         [ 0.6666, -0.2983,  0.0922, -1.0471,  1.2768,  1.1265,  1.0946,
           1.4914,  0.7205, -0.4837, -0.1287, -0.0737,  0.7687, -0.5311,
           1.4159, -1.6555],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000]],

        [[-1.5902, -1.0678,  1.8357,  0.0030, -1.1217, -1.3159, -0.6406,
          -0.1677,  1.0418, -0.7997,  0.0218, -0.5721, -0.9313, -0.4731,
          -0.4762,  0.1906],
         [-0.9226, -0.6351, -0.2052, -0.8969, -0.4510, -0.4629, -0.9043,
           0.7516,  0.4243, -1.1716,  0.5445, -0.0452, -1.0431,  0.0052,
          -0.2020,  0.2791],
         [ 0.8080, -1.1421, -1.2642,  0.5870, -0.5482,  0.1853,  1.5595,
          -0.2276, -0.6175,  1.0407,  1.5567,  0.9807, -1.3606,  1.1637,
          -1.8081, -1.4988],
         [-1.1202, -1.4404,  1.2313,  0.7874,  0.2202,  0.5783,  1.5642,
          -1.4807, -1.0937,  0.7240,  0.6155,  0.2237, -1.6044,  0.9075,
          -1.3431,  0.7700],
         [ 0.4308, -0.4005,  0.5008,  1.2831,  0.5751,  0.3695, -0.7355,
          -1.5796,  1.5674, -1.8740, -1.0547,  0.6519, -0.7017, -0.7984,
           0.0731, -0.9610],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000]]], grad_fn=<NativeBatchNormBackward0>)

In [17]:
class BatchNorm(torch.nn.BatchNorm1d):
    """ BatchNorm but with an optional bias and padding to support variable input length. PyTorch doesn't support simply bias=False """

    def __init__(self, num_features, bias, mask: int=0, *args, **kwargs): #arg names need to be identical to torch argnames for quantization support
        self.num_features = num_features
        self.mask = mask
        super().__init__(num_features, *args, **kwargs)

    def forward(self, input, *args, **kwargs):
        #dirty workaround to avoid runtimeerrors by adding a padding if the input is smaller than the feature length
        #padding does not artificially lower mean as normalization is performed along the word embeddings
        n,c,l = input.size()
        if c < self.num_features:
            #input tensor should always be three dimensional
            input = torch.cat((input, torch.full((n, self.num_features - c, l), self.mask)), dim=1)
        input = super().forward(input, *args, **kwargs)
        #remove padding 
        index = torch.tile(torch.arange(c).reshape(c,1), (n,1,l))
        return torch.gather(input=input, dim=1, index=index)

In [22]:
b,c,e = 12,64,256
batch = torch.rand(b,4,e)
bn = BatchNorm(c, bias=False, mask = -1)

In [23]:
padded_norm = bn(batch)

In [24]:
norm = torch.nn.BatchNorm1d(4)(batch)

In [27]:
padded_norm == norm

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        ...,

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, Tr

In [4]:
@dataclass
class Meta():
    num_tokens: int = 0
    other: str = ""

In [3]:
Meta(3).num_tokens = 7

/home/mabot004/eki-transformer-dev/qtransform/eki/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import os
#test if huggingface datasets can be created from text files
import datasets

BASEDIR = '/home/mabot004/.qtransform/datasets/files/shakespeare/untokenized/'
#number of rows depends on the amount of files
files = [os.path.join(BASEDIR, 'shakespeare.txt'), os.path.join(BASEDIR, 'shakespeare_2.txt')]
#does the same as huggingface mapping but now with files
def gen_text():
    for filename in files:
        with open(filename, 'r') as file:
            yield {"text": file.read()}

#chunk size from config, default 100
def chunk_examples(examples):
                #splits the text of each row into chunks of length chunk_length. currently it is only used
                #for character tokenization to avoid feeding large samples to the tokenizer
    chunk_length = 100
                #perform tokenization on a handful of characters at a time
                #from: https://huggingface.co/docs/datasets/process#split-long-examples            
    chunks = []
    
    for sentence in examples["text"]:
        new_chunks = [sentence[i:i + chunk_length] for i in range(0, len(sentence), chunk_length)]
        chunks.extend(new_chunks)
    return {"chunks": chunks}
from tiktoken import get_encoding
tokenizer = get_encoding("gpt2")

/home/mabot004/eki-transformer-dev/qtransform/eki/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
shakespeare = datasets.Dataset.from_generator(gen_text)

Generating train split: 2 examples [00:00, 32.58 examples/s]


In [6]:
chunks = shakespeare.map(chunk_examples, batched=True, remove_columns = "text")

Map: 100%|██████████| 2/2 [00:00<00:00, 20.24 examples/s]


In [18]:
from tqdm import tqdm
msg = 'ok'
for i in tqdm(range(100), desc=f'{msg}'):
    msg = str(i)

ok: 100%|██████████| 100/100 [00:00<00:00, 842229.72it/s]


In [16]:
rotten_tomatoes["train"].shard(num_shards=1000, index=0, contiguous = True)

Dataset({
    features: ['text', 'label'],
    num_rows: 9
})

In [26]:
rotten_tomatoes.select_columns("text")

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1066
    })
})

In [3]:
rotten_tomatoes = datasets.load_dataset('rotten_tomatoes')

In [10]:
#error occurs because the splits have more than one feature and this function changes the amount of samples in each split of one feature without changint the other
#so: 5 samples, 2 features. after mapping: text has 10 samples, other feature still has 5 features
#from: https://github.com/huggingface/datasets/issues/1817#issuecomment-774066254
rt_chunks = datasets.concatenate_datasets(rotten_tomatoes.select_columns("text").map(chunk_examples, batched=True, remove_columns = "text").values())

In [13]:
rt_chunks

Dataset({
    features: ['input_ids'],
    num_rows: 17458
})

In [12]:
rt_chunks = rt_chunks.map(
    #map function expects dictionary or dataset object, tokenize function returns list of tokens (integers)
    lambda batch: {"input_ids": [tokenizer.encode(x) for x in batch["chunks"]]}, 
    batched=True, 
    remove_columns = "chunks",
    #num_proc=os.cpu_count()//2 if cfg.encoding != 'character' else 1 
    desc="tokenizing the dataset from chunks")

tokenizing the dataset from chunks: 100%|██████████| 17458/17458 [00:00<00:00, 42685.95 examples/s]


In [14]:
rt_chunks.save_to_disk('/home/mabot004/custom_hf_datasets/')

Saving the dataset (1/1 shards): 100%|██████████| 17458/17458 [00:00<00:00, 437870.21 examples/s]


In [7]:
rt_chunks

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 13952
    })
    validation: Dataset({
        features: ['input_ids'],
        num_rows: 1761
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 1745
    })
})

In [36]:
tokenizer.decode(rt_chunks["train"]["input_ids"][0])

'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash eve'

In [20]:

#https://huggingface.co/docs/datasets/create_dataset#from-local-files
shakespeare = datasets.Dataset.from_generator(gen_text)
shakespeare = shakespeare.map(chunk_examples, batched=True, remove_columns = "text")
shakespeare = shakespeare.map(
    #map function expects dictionary or dataset object, tokenize function returns list of tokens (integers)
    lambda batch: {"input_ids": [tokenizer.encode(x) for x in batch["chunks"]]}, 
    batched=True, 
    remove_columns = "chunks",
    #num_proc=os.cpu_count()//2 if cfg.encoding != 'character' else 1 
    desc="tokenizing the dataset from chunks")

In [23]:
tokenizer.decode(np.concatenate(shakespeare[:3]["input_ids"]))


"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us"

In [ ]:
def write_memmap(memmap, start, end, data):
    memmap[start:end] = data

In [39]:
def gen_text():
    for i in range(163):
        yield {"text": i}

test_threading = datasets.Dataset.from_generator(gen_text)

Generating train split: 163 examples [00:00, 31912.97 examples/s]


In [52]:
test_threading.rename_column("text", "chunks")

Dataset({
    features: ['chunks'],
    num_rows: 163
})

In [51]:
test_threading.shard(num_shards=30, index=17)

Dataset({
    features: ['text'],
    num_rows: 5
})

In [43]:
import threading
num_threads = 3 #os.cpu_count // 2
batch_size = 30
num_samples = len(test_threading)
# 163 // 30 shards
# -> 3 threads, each having a batch size of 30 samples
# dataset has 163 samples -> each thread should have around 50-60 samples max
# -> divide samples of dataset with num_threads
# -> each thread should have the entire dataset as an arg, but split differently
# range of splitting should be specified as an arg in thread -> index arg in parameter

In [38]:
#why should you use multithreading? the writing process is I/O based
#if anything, the amount of write requests increases with the amount of threads
memmap = np.memmap('test', mode='w+', shape=(163,), dtype=np.int64)

In [ ]:

data = [x for x in range(163)]
def write(memmap, index):
    memmap[] = data[]

#threads = [threading.Thread() for x in range(3)]

In [2]:
import time
for i in range(10):
    time.sleep(1)

In [ ]:
#test if memmap file can be removed when catching sigint
import signal

def delete_memmap():
    pass

signal.signal(signal.SIGINT, delete_memmap)


In [6]:
try:
    int("abcd")
except Exception as e:
    print(str(e))

invalid literal for int() with base 10: 'abcd'


In [10]:
class Foo():
    def __init__(self):
        self.a = 10
    def function(self):
        def other():
            print(self.a)
        other()

Foo().function()

10


In [ ]:
if not exists(self.tokenized_file):
    hf_tokenized_dir = join(tokenized_dir, huggingface_tokens_dir)
    if not exists(hf_tokenized_dir):
        #perform tokenization
        pass
        dataset_splits = None
    else:
        dataset_splits = load_dataset(hf_tokenized_dir)
    #perform memmap writing
    pass

In [22]:
import torch
import numpy as np

#no high memory usage as memmap values are lazily loaded, only overhead is the pages (around 5MB per memmap )
memmap = np.memmap('/home/mabot004/.qtransform/datasets/huggingface/openwebtext/tokenized/gpt2/openwebtext-float32.bin', dtype=np.float32, mode='r')
memmap2 = np.memmap('/home/mabot004/.qtransform/datasets/huggingface/openwebtext/tokenized/gpt2/openwebtext-float32.bin', dtype=np.float32, mode='r')
memmap3 = np.memmap('/home/mabot004/.qtransform/datasets/huggingface/openwebtext/tokenized/gpt2/openwebtext-float32.bin', dtype=np.float32, mode='r')
memmap4 = np.memmap('/home/mabot004/.qtransform/datasets/huggingface/openwebtext/tokenized/gpt2/openwebtext-float32.bin', dtype=np.float32, mode='r')

In [2]:
import numpy as np
from typing import List, Tuple
import torch
from torch.utils.data import Dataset, DataLoader
from logging import getLogger
import os
log = getLogger('memmapDataset')
class MemmapDataset(Dataset):
    
    #TODO: is dtype necessary? since each file only contains the ids of tokens, not the actual embeddings
    def __init__(self, token_file: str, dtype: np.dtype, block_size: int, start: float=0.0, end: float = 1.0):
        """
            Creates a dataset which loads a numpy array from a file. 
            Slices of the dataset can be retrieved with the start and end parameters. 
            They specify the starting and ending range of the dataset in percent.
        """
        super().__init__()
        if not isinstance(start, float) or start < 0.0 or start >= 1.0:
            log.error(f'Invalid starting range for dataset ({start})')
            raise KeyError()
        if not isinstance(end, float) or end <= 0.0 or end > 1.0:
            log.error(f'Invalid ending range for dataset ({end})')
            raise KeyError()
        if not isinstance(dtype, np.dtype): #np.dtype("dtype") or np.<dtype> e.g.: np.dtype("float32") or np.float32
            try:
                dtype = np.dtype(dtype) #not an instance (np.float32)
            except TypeError as e:
                log.error(e)
                raise TypeError
        self.block_size = block_size
        if self.block_size <= 0:
            log.error(f'Block size of 0 is invalid.')
            raise ValueError()
        log.info(f"Attempting to retrieve tokenized dataset under \"{token_file}\"")
        self.token_file = token_file
        self.dtype = dtype
        #the method of retrieving the byte size is somewhat inspired from the stackoverflow article
        #https://stackoverflow.com/questions/19599864/easy-way-of-getting-number-of-bits-from-a-numpy-type
        self.bytes = self.dtype.itemsize
        amnt_tokens = os.path.getsize(self.token_file) / self.bytes
        if amnt_tokens % 1 != 0.0:
            log.error(f'The amount of tokens is supposed to be a whole number, but it is {amnt_tokens}. Maybe due to a wrong datatype?')
            raise ValueError()
        offset = int(amnt_tokens * start)
        #rounding to the nearest multiplicative of datatype to make sure not to read half a token too much
        offset -= offset % self.bytes
        log.debug(f'Offset is {offset}, start is {start}, end is {end}')
        #skip the first start * amnt_tokens and the last amnt_tokens * end items
        log.debug(f'Tokenized file has {amnt_tokens} tokens of datatype: {dtype}. Attempting to start at token: {offset}')
        self.data = np.memmap(self.token_file, dtype=self.dtype, mode='r', offset=offset)[:int(amnt_tokens * end)]
        if len(self.data) < self.block_size:
            log.error(f'Loaded data has less tokens than block size {self.block_size} for starting range {start} and ending range {end}. Maybe check size of splits?')
            raise ValueError()
        log.info(f'Loaded data has {len(self.data)} tokens.')
        #log.debug(f'all unique tokens in dataset: {set(self.data)}, length: {len(set(self.data))}')
        self.length = len(self.data)

    def __len__(self):
        return self.length
    
    #the dataloader works with batch sizes, dataset only works with indices
    def __getitem__(self, index) -> Tuple[torch.Tensor, torch.Tensor]:
        """
            Returns inputs and labels. Called when iterating with e.g. a dataloader.
        """
        if index < 0:
            index += len(self)
        #lower index to make sure that block_size elements are always retrieved
        if index + self.block_size > len(self) - 1:
            index = self.length - self.block_size - 2
        offset = index + self.block_size
        #fixed dtype as torch embeddings need int64 tensor to work
        inputs: torch.Tensor = torch.from_numpy(self.data[index:offset].astype(np.int64))
        #labels are always the following word for each word within the context
        labels : torch.Tensor = torch.from_numpy(self.data[index +1:offset+1].astype(np.int64))
        return inputs, labels

In [16]:
#token_file: str, dtype: np.dtype, block_size: int, start: float=0.0, end: float = 1.0
memmap_ds = MemmapDataset(
    token_file='/home/mabot004/.qtransform/datasets/huggingface/openwebtext/tokenized/gpt2/openwebtext-float32.bin',
    dtype=np.float32,
    block_size=64,
    start=0.0,
    end=0.3
)
len(memmap_ds)

2709600997

In [19]:
dataloader = DataLoader(memmap_ds, batch_size=12, num_workers=8)

In [14]:
next(iter(dataloader))

[tensor([[13924,    12,   559,    12, 35784,    11, 25051,   357, 18474,     8,
           1377, 45591,  4970,    11,  1319, 44556,   287,  2356,   290, 44787,
            379,  1204,    11,  7342,  7519,   290, 20669,  2513,  1497,   422,
            257,  2214,  4436,  3217,  1755,   706,   257, 21402,  3315,  1074,
          23724,   262,  1989,    11,  2282,   340,   373,  5213,   546,  2324,
             13,   198,   198,   464,  2551,  1364,  8100,  5953,  8366, 34428,
            298,  2986, 33708, 42095],
         [   12,   559,    12, 35784,    11, 25051,   357, 18474,     8,  1377,
          45591,  4970,    11,  1319, 44556,   287,  2356,   290, 44787,   379,
           1204,    11,  7342,  7519,   290, 20669,  2513,  1497,   422,   257,
           2214,  4436,  3217,  1755,   706,   257, 21402,  3315,  1074, 23724,
            262,  1989,    11,  2282,   340,   373,  5213,   546,  2324,    13,
            198,   198,   464,  2551,  1364,  8100,  5953,  8366, 34428,   298,
 

In [20]:
for i, data in enumerate(dataloader):
    if i == 10:
        break
    input, labels = data
    print(f'{input.size()}, {labels.size()}')

torch.Size([12, 64]), torch.Size([12, 64])
torch.Size([12, 64]), torch.Size([12, 64])
torch.Size([12, 64]), torch.Size([12, 64])
torch.Size([12, 64]), torch.Size([12, 64])
torch.Size([12, 64]), torch.Size([12, 64])
torch.Size([12, 64]), torch.Size([12, 64])
torch.Size([12, 64]), torch.Size([12, 64])
torch.Size([12, 64]), torch.Size([12, 64])
torch.Size([12, 64]), torch.Size([12, 64])
torch.Size([12, 64]), torch.Size([12, 64])


In [45]:
# Process.memory_info is expressed in bytes, so convert to megabytes
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

RAM used: 324.37 MB


In [7]:
#test progress bar and other stdout stuff
from tqdm import tqdm
import time
for i, data in tqdm(enumerate(range(10)), desc='test progress bar and other stdout stuff'):
    print(data)
    time.sleep(0.5)

test progress bar and other stdout stuff: 0it [00:00, ?it/s]

0


test progress bar and other stdout stuff: 1it [00:00,  2.00it/s]

1


test progress bar and other stdout stuff: 2it [00:01,  2.00it/s]

...

test progress bar and other stdout stuff: 8it [00:04,  1.99it/s]

8


test progress bar and other stdout stuff: 9it [00:04,  1.99it/s]

9


test progress bar and other stdout stuff: 10it [00:05,  1.99it/s]


In [53]:
c = 2 #2 words
prompt = torch.randint(200, (3, c, 4))
prompt

tensor([[[155, 152, 196,  34],
         [ 61,  80,   4, 114]],

        [[179, 106, 184, 132],
         [ 79, 105,  36,   6]],

        [[165,  82,  48,  48],
         [  4, 107, 119, 187]]])

In [55]:
torch.arange(c).reshape((c,1)).repeat((3,1,4))

tensor([[[0, 0, 0, 0],
         [1, 1, 1, 1]],

        [[0, 0, 0, 0],
         [1, 1, 1, 1]],

        [[0, 0, 0, 0],
         [1, 1, 1, 1]]])

In [56]:
torch.tile(torch.arange(c).reshape((c,1)), (3,1,4))

tensor([[[0, 0, 0, 0],
         [1, 1, 1, 1]],

        [[0, 0, 0, 0],
         [1, 1, 1, 1]],

        [[0, 0, 0, 0],
         [1, 1, 1, 1]]])

In [2]:
#testing batchnorm quant
#https://github.com/Xilinx/brevitas/issues/542
#https://github.com/Xilinx/brevitas/issues/363
import torch
import brevitas.nn as qnn
import torchvision
from torch import nn

/home/mabot004/eki-transformer-dev/qtransform/eki/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/home/mabot004/eki-transformer-dev/qtransform/eki/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#test if quantized layers having return_quant_tensor set to True are compatible with torch operations 
quant_tensor_linear = qnn.QuantLinear(1,1,True,return_quant_tensor=True)
quant_tensor_linear(torch.Tensor(8,1)) #works

In [35]:
from omegaconf import DictConfig

In [9]:
#debug loading quantized checkpoint
CHECKPOINT = '/home/mabot004/eki-transformer-dev/qtransform/outputs/models/GPT_2024-01-17_08:30:49__epoch:1'
#doesnt work since qtransform.dataset cannot be found
#but module info about tokenizers is not saved in checkpoint, only their names
checkpoint = torch.load(CHECKPOINT)
checkpoint.keys()

dict_keys(['model_state_dict', 'optimizer_state_dict', 'epoch', 'model_cfg', 'tokenizer_cfg', 'metrics'])

In [32]:
#check if info about quant params are even saved within checkpoint
import re
keys = checkpoint["model_state_dict"].keys()
#quant param that exists within checkpoint: 
#transformer.layer.0.mlp.active.act_quant.fused_activation_quant_proxy.tensor_quant.scaling_impl.value 
weights_and_biases = list(filter(lambda x: re.search(r'.+\.(weight|bias)$', x), keys))
def find(x):
    if not re.search(r'.+\.(weight|bias)$', x):
        return x
other_keys = list(filter(find, keys))
len(keys) == len(weights_and_biases) # not only weights and biases in state dict
#only scaling_impl is saved in state dict
#no multiheadattention though
#in gpt quant config, every single layer has a quantizer (most commonly Int8WeightPerTensorFloat)
#that quantizer has ScalingImplType STATS
#the layers with scaling_impl had an activation quantizer named Int8ActPerTensorFloat
#it had the ScalingImplType PARAMETER_FROM_STATS
other_keys

['transformer.layer.0.attn.attn_mask',
 'transformer.layer.0.mlp.active.input_quant.fused_activation_quant_proxy.tensor_quant.scaling_impl.value',
 'transformer.layer.0.mlp.active.act_quant.fused_activation_quant_proxy.tensor_quant.scaling_impl.value',
 'transformer.layer.1.attn.attn_mask',
 'transformer.layer.1.mlp.active.input_quant.fused_activation_quant_proxy.tensor_quant.scaling_impl.value',
 'transformer.layer.1.mlp.active.act_quant.fused_activation_quant_proxy.tensor_quant.scaling_impl.value']

In [46]:
#test if scaling_impl params exist within model
test_mha = qnn.QuantMultiheadAttention(num_heads=2, embed_dim=256)
#simulate some learning steps for param
test_mha.v_quant.act_quant.fused_activation_quant_proxy.tensor_quant.scaling_impl.value = torch.nn.Parameter(torch.tensor(3.1415))
test_mha.v_quant.act_quant.fused_activation_quant_proxy.tensor_quant.scaling_impl.value

In [57]:
from brevitas.quant import scaled_int

In [58]:
#test which brevitas layers return qparams in state_dict
qnn.QuantLinear(1,1,True,input_quant=scaled_int.Int8ActPerTensorFloat).state_dict()

OrderedDict([('weight', tensor([[0.9874]])), ('bias', tensor([-0.8623]))])

In [61]:
qnn.QuantIdentity(act_quant=scaled_int.Int8ActPerTensorFloat).state_dict()

OrderedDict()

In [62]:
qnn.QuantReLU(act_quant=scaled_int.Int8ActPerTensorFloat).state_dict()

OrderedDict()

In [66]:
#for some reason, qparams are saved within this model
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.active  = nn.ReLU() #or GELU
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x): 
        x = self.c_fc(x)
        x = self.active(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

from dataclasses import dataclass
class Config():
    n_embd: int = 256
    dropout: bool = True
    bias: bool = True

mlp = MLP(Config())


In [68]:
mlp.state_dict().keys()

odict_keys(['c_fc.weight', 'c_fc.bias', 'c_proj.weight', 'c_proj.bias'])

In [50]:
torch.save(test_mha.state_dict(), 'mha.chpt')

In [54]:
#v_quant etc. not appearing within state_dict
test_mha.state_dict().keys()

odict_keys(['in_proj.weight', 'in_proj.bias', 'out_proj.weight', 'out_proj.bias'])

In [28]:
re.search(r'(?!hallo|welt).*$', "hallo")

<re.Match object; span=(1, 5), match='allo'>

In [4]:
#check if storing checkpoints of quantized models even is working
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.network = torch.nn.ModuleDict(dict(
            wte = qnn.QuantEmbedding(32, 128),
            pos = qnn.QuantEmbedding(16, 128),
            logic = nn.ModuleDict(dict(
                layer1 = qnn.QuantLinear(128, 16, True),
                layer2 = qnn.QuantLinear(16,1, True))
            )
        ))
    def forward(self, x):
        embd = self.network.wte(x)
        b,t = x.size()
        pos = torch.arange(0, t, dtype=torch.long).unsqueeze(0) # shape (1, t)
        pos = self.network.pos(pos)
        output = embd + pos
        for name, layer in self.network.logic.items():
            output = layer(output)
        return output

In [5]:
model = Model()
model(torch.randint(32, (1,16)))

/home/mabot004/eki-transformer-dev/qtransform/eki/lib/python3.10/site-packages/torch/_tensor.py:1362: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at ../c10/core/TensorImpl.h:1900.)
  return super().rename(names)


tensor([[[-0.3944],
         [ 0.2287],
         [-0.5937],
         [-0.8445],
         [ 0.4049],
         [-0.1961],
         [ 0.2558],
         [ 0.5325],
         [-0.2270],
         [ 0.0485],
         [-0.5637],
         [ 0.1862],
         [ 0.7595],
         [-0.2511],
         [ 0.1841],
         [-0.3207]]], grad_fn=<ViewBackward0>)

In [6]:
#doesnt work, Quantizer cannot be found in brevitas.inject
#why are they being searched for in inject if they are in brevitas.quant.scaled_int
torch.save(model, 'quantized_test') 

PicklingError: Can't pickle <class 'brevitas.inject.Int8WeightPerTensorFloat'>: attribute lookup Int8WeightPerTensorFloat on brevitas.inject failed

In [8]:
#test class attributes
class DeviceSingleton:
    """
        Boilerplate class which contains the device used for the entire process. The reason why a class is created is in order to monitor when changes
        to the device are going to occur by using the property decorator. Currently, changes will be allowed and logged.
    """
    _device = None
    def __init__(self):
        
        pass

    @property
    def device():
        return _device

    @device.setter
    def device(new_device):
        match new_device:
            case 'cuda':
                new_device = 'cuda' if cuda.is_available() else 'cpu'
            case 'gpu':
                new_device = 'cuda' if cuda.is_available() else 'cpu'
            case 'mps':
                new_device = 'mps' if backends.mps.is_available() else 'cpu'
            case 'cpu':
                new_device = 'cpu'
            case _:
                print(f'Device {new_device} not recognized. Using default: CPU')
                new_device = 'cpu'
        _device = device(new_device)
        print(f'Using device: {new_device}')

In [6]:
DeviceSingleton.device = 'cpu'

In [9]:
singleton = DeviceSingleton()

In [10]:
singleton.device

TypeError: DeviceSingleton.device() takes 0 positional arguments but 1 was given

In [7]:
DeviceSingleton.device

'cpu'

In [38]:
def fuse_conv_and_bn(conv, bn):
	#
	# init
	fusedconv = torch.nn.Conv1d(
		conv.in_channels,
		conv.out_channels,
		kernel_size=conv.kernel_size,
		stride=conv.stride,
		padding=conv.padding,
		bias=True
	)
	#
	# prepare filters
	w_conv = conv.weight.clone().view(conv.out_channels, -1)
	w_bn = torch.diag(bn.weight.div(torch.sqrt(bn.eps+bn.running_var)))
	fusedconv.weight.copy_( torch.mm(w_bn, w_conv).view(fusedconv.weight.size()) )
	#
	# prepare spatial bias
	if conv.bias is not None:
		b_conv = conv.bias
	else:
		b_conv = torch.zeros( conv.weight.size(0) )
	b_bn = bn.bias - bn.weight.mul(bn.running_mean).div(torch.sqrt(bn.running_var + bn.eps))
	fusedconv.bias.copy_( torch.matmul(w_bn, b_conv) + b_bn )
	#
	# we're done
	return fusedconv


torch.set_grad_enabled(False)
batch_size = (16, 64, 256)
x = torch.randn(16, 64, 256)

net = torch.nn.Sequential(
    torch.nn.Conv1d(64, 64, kernel_size=(256,256)),
    torch.nn.BatchNorm1d(64)
)
y1 = net.forward(x)
fusedconv = fuse_conv_and_bn(net[0], net[1])
y2 = fusedconv.forward(x)
d = (y1 - y2).norm().div(y1.norm()).item()
print("error: %.8f" % d)

error: 0.49767026


In [74]:
cv1 = qnn.QuantLinear(5,5,bias=True)
cv1_copy = qnn.QuantLinear(5,5,bias=True)
cv1_copy.load_state_dict(cv1.state_dict())
bn1 = torch.nn.BatchNorm1d(5)
qnn.utils.merge_bn(cv1, bn1)
input = torch.Tensor(2,5)

In [78]:
cv1 is cv1_copy

False

In [75]:
cv1(input)

tensor([[ 3.0883e-01, -9.2026e-03,  3.9793e-01,  3.7391e-01,  4.2723e-01],
        [-3.9785e+20,  2.8513e+20, -5.6362e+19,  2.4866e+20,  3.8127e+20]],
       grad_fn=<AddmmBackward0>)

In [77]:
#output is the same without batchnorm, why?
output = cv1_copy(input)
output

tensor([[ 3.0883e-01, -9.2026e-03,  3.9794e-01,  3.7391e-01,  4.2723e-01],
        [-3.9785e+20,  2.8513e+20, -5.6362e+19,  2.4866e+20,  3.8127e+20]],
       grad_fn=<AddmmBackward0>)

In [81]:
output = bn1(input)
cv1_copy(output)

tensor([[ 0.3088, -0.0092,  0.3979,  0.3739,  0.4272],
        [ 0.3088, -0.0092,  0.3979,  0.3739,  0.4272]],
       grad_fn=<AddmmBackward0>)

In [ ]:
tensor = torch.randint(30, (3,5,20)).to(dtype=torch.float32) / 10

In [71]:
torch.torch.nn.BatchNorm1d(5)(qnn.QuantConv1d(5,5,kernel_size=3)(tensor))

tensor([[[-1.5470e+00,  1.5270e-01, -2.5464e-01,  2.9082e-01,  1.2001e+00,
           6.7265e-01,  7.3291e-01, -1.8250e-01, -1.2105e+00, -1.0728e+00,
           3.1590e-01,  2.5334e+00, -1.3411e-01,  2.5901e+00,  6.9120e-02,
          -5.1725e-01,  1.9493e-01,  2.2703e+00],
         [-3.4492e-01, -9.6099e-01, -9.2788e-01, -5.6099e-01, -2.0823e+00,
           8.8297e-01,  4.6034e-01,  9.3609e-01,  1.8312e+00, -8.3214e-01,
          -1.0253e+00, -1.3361e+00, -1.3721e+00,  4.9575e-01, -6.1378e-01,
           3.7313e-01, -1.6607e+00, -7.7247e-01],
         [ 1.8919e+00,  8.4047e-01,  5.7258e-01,  3.1601e-01,  1.4367e-01,
          -4.8590e-01,  7.8809e-01,  5.6231e-01, -9.9302e-01, -5.6623e-01,
          -2.1978e-01, -8.2209e-01, -2.8324e-02,  9.5371e-01,  9.6952e-02,
          -6.7169e-01, -8.7423e-02, -4.8495e-02],
         [-2.3595e+00, -4.5535e-01,  1.1663e+00,  1.6639e+00,  5.8315e-01,
          -4.0086e-01, -4.8103e-01, -2.0247e+00,  4.6665e-01,  2.1141e-01,
           2.7884e-02,  1

In [74]:
cv1(tensor)

tensor([[[ 1.5805,  1.6874,  1.4232,  0.4953,  0.3392,  1.6921,  0.4289,
           0.6056,  1.4143, -0.3349,  0.7054,  1.2688,  0.7755,  0.8200,
           0.8781,  1.1979,  0.5057,  0.3852],
         [-0.6557,  0.1111,  0.1168,  0.1569, -0.1044, -0.7017, -0.4960,
          -0.0382, -0.2186,  0.1632,  0.3991, -0.2319, -0.1412, -1.0671,
          -0.2227, -0.4349,  0.0598, -0.0862],
         [-1.8093, -1.3581, -1.4218, -1.9729, -2.4638, -1.5891, -2.7596,
          -1.0707, -0.9489, -1.2964, -0.4576, -2.2601, -2.3817, -2.9702,
          -2.1696, -1.4183, -1.6867, -1.5476],
         [ 0.0087,  0.9669,  0.2861, -0.0195,  1.7805, -0.1940,  1.5801,
           0.4090,  0.1231, -0.3474,  0.4677,  0.5724,  0.8208,  1.0850,
           1.3689,  0.1611,  0.5700, -0.0545],
         [-1.7327, -0.0812, -0.3896, -1.2272, -0.7673, -1.1279, -1.3454,
          -0.1006,  0.1854, -0.1570, -0.0988, -0.2189, -0.5590, -1.7284,
          -0.0418, -0.0416,  0.0663, -0.0783]],

        [[ 1.6639,  0.7003,  1.20

In [35]:
m = torch.nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)
output.size()

torch.Size([20, 33, 24])

In [3]:
#conv1d and batchnorm1d merge

qnn.quant_layer.merge_bn

<function brevitas.nn.utils.merge_bn(layer, bn, output_channel_dim=0)>

In [16]:
tensor = torch.rand((3,6,9))
tensor

tensor([[[0.9917, 0.4984, 0.6176, 0.5039, 0.8158, 0.8521, 0.0155, 0.1858,
          0.8048],
         [0.1621, 0.4298, 0.3947, 0.5427, 0.8238, 0.9419, 0.7478, 0.4333,
          0.0647],
         [0.0897, 0.2927, 0.9780, 0.6710, 0.0377, 0.8199, 0.1301, 0.8592,
          0.8216],
         [0.2074, 0.6790, 0.2042, 0.7838, 0.5414, 0.5088, 0.8481, 0.2490,
          0.1760],
         [0.0197, 0.6737, 0.1897, 0.2794, 0.4024, 0.3306, 0.8610, 0.8641,
          0.6871],
         [0.7651, 0.4413, 0.9831, 0.4328, 0.2344, 0.0799, 0.4901, 0.1151,
          0.9380]],

        [[0.4503, 0.5180, 0.3012, 0.7354, 0.2637, 0.9073, 0.9226, 0.7925,
          0.0674],
         [0.9067, 0.1654, 0.9186, 0.1072, 0.0438, 0.4049, 0.1374, 0.3990,
          0.6381],
         [0.3767, 0.8549, 0.5588, 0.2489, 0.2599, 0.6461, 0.5800, 0.1559,
          0.0832],
         [0.9381, 0.2192, 0.7259, 0.7615, 0.1411, 0.1472, 0.9268, 0.6733,
          0.9049],
         [0.1468, 0.8668, 0.3151, 0.5401, 0.4347, 0.5541, 0.0995, 0.

In [17]:
#normalized values along second dimension, meaning: along sentences
#are 
torch.nn.BatchNorm1d(6)(tensor)

tensor([[[ 1.5546, -0.3177,  0.1346, -0.2969,  0.8870,  1.0246, -2.1506,
          -1.5043,  0.8454],
         [-0.9784, -0.1143, -0.2279,  0.2499,  1.1573,  1.5384,  0.9118,
          -0.1030, -1.2930],
         [-1.5225, -0.7464,  1.8741,  0.7002, -1.7215,  1.2696, -1.3680,
           1.4198,  1.2762],
         [-1.0571,  0.6570, -1.0687,  1.0382,  0.1571,  0.0385,  1.2718,
          -0.9059, -1.1712],
         [-1.5085,  0.9969, -0.8575, -0.5138, -0.0426, -0.3174,  1.7148,
           1.7266,  1.0484],
         [ 1.0773, -0.1349,  1.8936, -0.1669, -0.9097, -1.4882,  0.0477,
          -1.3565,  1.7249]],

        [[-0.5003, -0.2434, -1.0663,  0.5820, -1.2088,  1.2342,  1.2923,
           0.7987, -1.9539],
         [ 1.4249, -0.9680,  1.4631, -1.1558, -1.3605, -0.1948, -1.0582,
          -0.2139,  0.5578],
         [-0.4253,  1.4034,  0.2712, -0.9140, -0.8717,  0.6049,  0.3522,
          -1.2693, -1.5475],
         [ 1.5987, -1.0141,  0.8276,  0.9569, -1.2980, -1.2758,  1.5576,
       

In [21]:
tensor[0][0].mean()

tensor(0.5873)

In [6]:
#tensor retains size, batchnorm essentially is a linear transformation to shift values to have a mean of 0 and a standard deviation of 1
torch.nn.BatchNorm1d(10)(torch.Tensor(3,10,16)).size()

torch.Size([3, 10, 16])

In [78]:
identity = qnn.QuantIdentity()
tensor = torch.Tensor(12,64,256)

In [83]:
tensor[0][0][0]

tensor(2.8026e-45)

In [84]:
output[0][0][0]

tensor(0.)

In [101]:
#test if quantidentity is a simple wrapper around a tensor that does nothing
#if so, it could be useful for merging with batchnorm
tensor = torch.Tensor(2,3,4)
print(tensor)
print("\n" + 30* "-" + "\n")
print(qnn.QuantIdentity()(tensor).isclose(tensor).all().item())

tensor([[[-1.1617e+35,  3.0907e-41, -1.5597e+37,  3.0907e-41],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  1.4013e-45,  0.0000e+00]],

        [[ 0.0000e+00,  0.0000e+00,  1.1351e-43,  0.0000e+00],
         [-1.5597e+37,  3.0907e-41, -3.0176e+34,  3.0907e-41],
         [ 0.0000e+00,  0.0000e+00,  1.4013e-45,  0.0000e+00]]])

------------------------------

False


In [81]:
output = identity(tensor)
output.size == tensor.size
output == tensor

tensor([[[False,  True, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [